# Directory Scanner AI Agent Example - Version 1

* This is a simple AI agent example code that lists files in a directory, reads the file content, and provides basic answers to questions the user asks about them.
* Utilizes an agent loop that receives input, decides on actions, executes the actions, and updates the LLM memory
* Version 1 agent can perform the following functions (tools): (1) list_files(): lists files in a current directory), (2) read_file(): returns file contents, (3) terminate(): ends the agent loop

In [7]:
from dotenv import load_dotenv
import os
import sys

# Check Python version (must be 3.10+)
if sys.version_info >= (3, 10):
    print(f"✓ Python version {sys.version_info.major}.{sys.version_info.minor} meets requirements")
else:
    print(f"✗ Python {sys.version_info.major}.{sys.version_info.minor} detected. Python 3.10+ required (see runtime.txt)")

# Load environment variables from .env file
env_loaded = load_dotenv()

if env_loaded:
    print("✓ .env file loaded successfully")
else:
    print("✗ .env file not found")

# Get the API key from environment
api_key = os.getenv('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = api_key

# Validate key exists and has correct format
if api_key and api_key.startswith('sk-'):
    print("✓ API key loaded successfully")
else:
    print("✗ API key not found or invalid format")
    

✓ Python version 3.12 meets requirements
✓ .env file loaded successfully
✓ API key loaded successfully


In [12]:
from litellm import completion #import completion function from litellm library which allows interacting with LLMs, allowing you to send/receive prompts
from typing import List, Dict
import json
import os
import re

In [14]:
# Helper function to generate LLM response
def generate_response(messages):
    response = completion(
        model="gpt-4o-mini",
        messages=messages
    )
    return response.choices[0].message.content



# Helper function to extract markdown code blocks
def extract_markdown_block(text: str, block_type: str) -> str:
    """Extract content from markdown code blocks like ```action"""
    pattern = f"```{block_type}\\s*\\n(.*?)\\n```"
    match = re.search(pattern, text, re.DOTALL)
    if match:
        return match.group(1).strip()
    # Fallback: try to find any JSON-like content
    return text.strip()

# Helper function to parse action from response
def parse_action(response: str) -> Dict:
    """Parse the LLM response into a structured action dictionary."""
    try:
        response = extract_markdown_block(response, "action")
        response_json = json.loads(response)
        if "tool_name" in response_json and "args" in response_json:
            return response_json
        else:
            return {"tool_name": "error", "args": {"message": "You must respond with a JSON tool invocation."}}
    except json.JSONDecodeError:
        return {"tool_name": "error", "args": {"message": "Invalid JSON response. You must respond with a JSON tool invocation."}}

# Tool functions
def list_files():
    try:
        files = [f for f in os.listdir('.') if os.path.isfile(f)]
        return files
    except Exception as e:
        return f"Error listing files: {str(e)}"

def read_file(file_name):
    try:
        with open(file_name, 'r') as f:
            return f.read()
    except Exception as e:
        return f"Error reading file: {str(e)}"

# Agent rules
agent_rules = [{
    "role": "system",
    "content": """
You are an AI agent that can perform tasks by using available tools.

Available tools:
- list_files() -> List[str]: List all files in the current directory.
- read_file(file_name: str) -> str: Read the content of a file.
- terminate(message: str): End the agent loop and print a summary to the user.

CRITICAL RULES:
1. EVERY response MUST contain ONLY an action block - no other text before or after
2. Do NOT provide conversational responses
3. Do NOT explain what you're doing outside the action block
4. After completing a task, immediately call terminate() with a summary

If a user asks about files, list them before reading.

REQUIRED response format (nothing else):
```action
{
    "tool_name": "insert tool_name",
    "args": {...fill in any required arguments here...}
}
```

Examples:
- To list files: ```action\n{"tool_name": "list_files", "args": {}}\n```
- To read a file: ```action\n{"tool_name": "read_file", "args": {"file_name": "example.txt"}}\n```
- To finish: ```action\n{"tool_name": "terminate", "args": {"message": "Task completed. Found 5 files."}}\n```
"""
}]

# Initialize memory with user task
user_task = input("What would you like the agent to do? ")
memory = [{"role": "user", "content": user_task}]

# Agent loop parameters
iterations = 0
max_iterations = 10

# Step 1: Define the Agent Loop
while iterations < max_iterations:
    # Construct the prompt
    '''Prompt construction: 
    - agent_rules contains pre-defined system instructs: 
        - Before agent begins the loop, it has rules that define its behavior
        - Agent rules are written as a system message that instructs LLM on how the agent should behave
        - These rules will be defined later in the code but executed during the agent loop
    - memory is record of all past interactions including user input and agent responses
    '''
    prompt = agent_rules + memory
    
    # Generate LLM response
    print("Agent thinking...")
    response = generate_response(prompt)
    print(f"Agent response: {response}") # optional but useful for debugging 
    
    # Parse response to determine action
    action = parse_action(response)
    
    # Router depending on which action LLM wants to take
    if action["tool_name"] == "list_files":
        result = {"result": list_files()}
    elif action["tool_name"] == "read_file":
        result = {"result": read_file(action["args"]["file_name"])}
    elif action["tool_name"] == "error":
        result = {"error": action["args"]["message"]}
    elif action["tool_name"] == "terminate":
        print(action["args"]["message"])
        break
    else:
        result = {"error": "Unknown action: " + action["tool_name"]}
    
    print(f"Action result: {result}")
    
    # 5. Update memory with response and results
    memory.extend([
        {"role": "assistant", "content": response},
        {"role": "user", "content": json.dumps(result)}
    ])
    
    # 6. Check termination condition
    if action["tool_name"] == "terminate":
        break
    
    iterations += 1

if iterations >= max_iterations:
    print("Agent reached maximum iterations.")

What would you like the agent to do?  Tell me about files in this directory


Agent thinking...
Agent response: ```action
{"tool_name": "list_files", "args": {}}
```
Action result: {'result': ['DirectoryScanner_AIAgent.ipynb', 'Environment and LiteLLM Setup.ipynb', 'requirements.txt', 'StarterAgentNotebook.ipynb', 'runtime.txt', 'README.MD', 'PythonFunctionGenerator.ipynb', 'agent.py']}
Agent thinking...
Agent response: ```action
{"tool_name": "read_file", "args": {"file_name": "README.MD"}}
```
Action result: {'result': '# Agent Examples\n\nThis directory includes examples of AI Agents and Agentic Tool & Environment Setup.\n\n## Files\n\n- `agent.py` - Core agent implementation and logic\n- `Environment and LiteLLM Setup.ipynb` - Environment configuration and LiteLLM setup\n- `StarterAgentNotebook.ipynb` - Starter notebook for agent development\n- `requirements.txt` - Python package dependencies\n- `runtime.txt` - Runtime environment specification\n'}
Agent thinking...
Agent response: ```action
{"tool_name": "terminate", "args": {"message": "Task completed. Rea